In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_list = drive.ListFile({'q': "'1-YZGx3TTna_Bm9R8q0I_jZ-AJ6neHVdY' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [ ]:
data = drive.CreateFile({'id': '1EyZUfGMTtzFG_f1uKd9qfRKfp7pSr0KA'})
data.GetContentFile('Airline_Arrivals_Cleaned1.csv')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter('ignore')
# set to unlimited column display:
pd.set_option('display.max_columns', None)
%matplotlib inline

In [ ]:
df = pd.read_csv('Airline_Arrivals_Cleaned1.csv',sep="\t")

In [ ]:
df.head()

In [ ]:
data = df[['ArrDelay','ActualElapsedTime','CRSArrTime','ArrTime','DepTime','CRSDepTime','Origin','DepDelay','Cancelled','Dest','Date','Distance','FlightNum','UniqueCarrier','Diverted']]

In [ ]:
missing_df = data.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variables','missing values']
missing_df['filling factor (%)'] = (data.shape[0]-missing_df['missing values'])/data.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop=True)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import RFE 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
# RFE is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through a coef_ attribute or through a feature_importances_ attribute. Then, the least important features are pruned from current set of features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached.
#SelectKBest, Function taking two arrays X and y, and returning a pair of arrays (scores, pvalues) or a single array with scores. Default is f_classif (see below “See also”). The default function only works with classification tasks.

k_filter = SelectKBest(f_regression,k=10)
estimator = SVR(kernel="linear")
pipe_lr = Pipeline([('scl', StandardScaler()),
                    ('PCA', PCA(n_components=2,svd_solver='full')),
                    ('SelectKBest', k_filter),
                    ('RFE', RFE(estimator, 5, step=1))
                   ])
pipe_lr.fit(df)